In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
train_datagen = ImageDataGenerator (rescale=1./255, shear_range=0.1,zoom_range=0.1, horizontal_flip=True)

In [33]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [34]:
training_set = train_datagen.flow_from_directory("C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\level\\training",target_size = (224, 224),batch_size = 10,class_mode= 'categorical')
test_set = test_datagen.flow_from_directory("C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\level\\validation",target_size = (224, 224),batch_size = 10,class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [35]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [36]:
imageSize=[224,224]
Vgg = VGG16(input_shape=imageSize+[3], weights='imagenet', include_top=False)

In [37]:
for layer in Vgg.layers:
    layer.trainable =False

In [38]:
X = Flatten()(Vgg.output)

In [39]:
prediction = Dense(3, activation='softmax')(X)

In [40]:
model=Model(inputs=Vgg.input,outputs=prediction)

In [41]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['acc'])

In [43]:
import sys
#fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=979//10,
  validation_steps=171//10)

C:\Users\ssdha\AppData\Local\Temp\ipykernel_16232\1270631010.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
97/97 [==============================] - 615s 6s/step - loss: 1.2952 - acc: 0.5511 - val_loss: 0.9781 - val_acc: 0.6176
Epoch 2/25
97/97 [==============================] - 469s 5s/step - loss: 0.7509 - acc: 0.7028 - val_loss: 1.1403 - val_acc: 0.6000
Epoch 3/25
97/97 [==============================] - 469s 5s/step - loss: 0.6111 - acc: 0.7771 - val_loss: 1.3184 - val_acc: 0.5588
Epoch 4/25
97/97 [==============================] - 475s 5s/step - loss: 0.4141 - acc: 0.8235 - val_loss: 1.3696 - val_acc: 0.5882
Epoch 5/25
97/97 [==============================] - 472s 5s/step - loss: 0.3886 - acc: 0.8493 - val_loss: 1.1504 - val_acc: 0.6059
Epoch 6/25
97/97 [==============================] - 462s 5s/step - loss: 0.2875 - acc: 0.8927 - val_loss: 1.3244 - val_acc: 0.6118
Epoch 7/25
97/97 [==============================] - 486s 5s/step - loss: 0.2268 - acc: 0.9205 - val_loss: 1.1829 - val_acc: 0.6294
Epoch 8/25
97/97 [==============================] - 476s 5s/step - loss: 0.1855 - a

In [44]:
model.save('level.h5')

In [51]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [52]:
model=load_model('level.h5')

In [53]:
#creating a function, which is used to predict, so need some preprocessing 
def detect (frame):
    img = cv2.resize(frame, (224,224)) #resizing the image to model trained image size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # uploaded image is in the form of BGR, so convert to RGB
    # scaling need to be done
    if(np.max(img)>1):
        img=img/255.0
    img= np.array([img]) #then to array fpormat 
    prediction = model.predict(img)
    print(prediction)
    label=["minor","moderate","severe"]
    preds =label [np.argmax(prediction)]
    return preds

In [54]:
import numpy as np

In [65]:
data="C:\\Users\\ssdha\\Downloads\\Car damage-20221030T082416Z-001\\Car damage\\level\\validation\\03-severe\\0048.JPEG"
image=cv2.imread(data)

In [66]:
print(detect(image))

1/1 [==============================] - 0s 180ms/step
[[4.0794319e-08 2.5020361e-05 9.9997497e-01]]
severe
